In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium

In [2]:
gdf = gpd.read_file('Lower_layer_Super_Output_Areas_(December_2021)_Boundaries_EW_BFC_(V10).shp')
gdf.rename(columns={'LSOA21CD': 'LSOA Code', 'LSOA21NM': 'LSOA Name'}, inplace=True)
gdf.set_index('LSOA Code', inplace=True)
gdf = gdf.to_crs("EPSG:4326")
gdf.head()


,FID,LSOA Name,LSOA21NMW,BNG_E,BNG_N,LAT,LONG,Shape__Are,Shape__Len,GlobalID,geometry
LSOA Code,,,,,,,,,,,
E01000001,1,City of London 001A,None,532123,181632,51.51817,-0.097150,129865.314476,2635.767993,c625aea8-6d73-4b2a-be76-4d5c44cad9f8,"POLYGON ((-0.09667 51.52027, -0.09666 51.52025..."
E01000002,2,City of London 001B,None,532480,181715,51.51883,-0.091970,228419.782242,2707.816821,52c878e9-ac68-4886-b4a8-fea9cd241a70,"POLYGON ((-0.08969 51.52069, -0.08973 51.52057..."
E01000003,3,City of London 001C,None,532239,182033,51.52174,-0.095330,59054.204697,1224.573160,b9d8faca-d489-478d-8ce6-acaf76186d7d,"POLYGON ((-0.09653 51.52295, -0.09647 51.52282..."
E01000005,4,City of London 001E,None,533581,181283,51.51469,-0.076280,189577.709503,2275.805344,15e1417d-537c-4845-9820-fc7596bd59b0,"POLYGON ((-0.07571 51.51575, -0.07542 51.51555..."
E01000006,5,Barking and Dagenham 016A,None,544994,184274,51.53875,0.089317,146536.995750,1966.092607,8a6c4ee0-c0ff-4736-9cfa-fb12a6d50da0,"POLYGON ((0.09122 51.53905, 0.09148 51.5389, 0..."


In [3]:
lad = pd.read_csv('LSOA LAD Lookup.csv')
lad.rename(columns={'LSOA21CD':'LSOA Code', 'LSOA21NM':'a', 'LSOA21NMW':'b', 'LAD23CD':'c', 'LAD23NM':'LAD Name', 'LAD23NMW':'d', 'ObjectId':'e'}, inplace=True)
lad = lad.drop(columns=['a', 'b', 'c', 'd', 'e'])
lad.set_index('LSOA Code', inplace=True)
lad.head()

,LAD Name
LSOA Code,
E01011949,Hartlepool
E01011950,Hartlepool
E01011951,Hartlepool
E01011952,Hartlepool
E01011953,Hartlepool


In [4]:
gdf = gdf.merge(lad[['LAD Name']], left_index=True, right_index=True)
gdf.head()
                

,FID,LSOA Name,LSOA21NMW,BNG_E,BNG_N,LAT,LONG,Shape__Are,Shape__Len,GlobalID,geometry,LAD Name
LSOA Code,,,,,,,,,,,,
E01000001,1,City of London 001A,None,532123,181632,51.51817,-0.097150,129865.314476,2635.767993,c625aea8-6d73-4b2a-be76-4d5c44cad9f8,"POLYGON ((-0.09667 51.52027, -0.09666 51.52025...",City of London
E01000002,2,City of London 001B,None,532480,181715,51.51883,-0.091970,228419.782242,2707.816821,52c878e9-ac68-4886-b4a8-fea9cd241a70,"POLYGON ((-0.08969 51.52069, -0.08973 51.52057...",City of London
E01000003,3,City of London 001C,None,532239,182033,51.52174,-0.095330,59054.204697,1224.573160,b9d8faca-d489-478d-8ce6-acaf76186d7d,"POLYGON ((-0.09653 51.52295, -0.09647 51.52282...",City of London
E01000005,4,City of London 001E,None,533581,181283,51.51469,-0.076280,189577.709503,2275.805344,15e1417d-537c-4845-9820-fc7596bd59b0,"POLYGON ((-0.07571 51.51575, -0.07542 51.51555...",City of London
E01000006,5,Barking and Dagenham 016A,None,544994,184274,51.53875,0.089317,146536.995750,1966.092607,8a6c4ee0-c0ff-4736-9cfa-fb12a6d50da0,"POLYGON ((0.09122 51.53905, 0.09148 51.5389, 0...",Barking and Dagenham


In [5]:
dep = pd.read_csv('LSOA Deprivation.csv')
new_cols = dep.loc[5].values
dep.columns = new_cols
dep['LSOA Code'] = dep['2021 super output area - lower layer'].str.split(' : ').str[0]
dep['LSOA Name'] = dep['2021 super output area - lower layer'].str.split(' : ').str[1]
dep = dep.drop(columns=['2021 super output area - lower layer'])
dep.set_index('LSOA Code', inplace=True)
dep = dep.iloc[7:-7]
dep.columns = ['Total Households','%','Not-N','Not-%','1-N','1-%','2-N','2-%','3-N','3-%','4-N','4-%','LSOA Name']
dep.drop('%', axis=1, inplace=True)
dep = dep.apply(lambda x: pd.to_numeric(x, errors='coerce') if x.name != "LSOA Name" else x)
dep['Deprived %'] = 100 - dep['Not-%']


gdf = gdf.merge(dep[['Deprived %']], left_index=True, right_index=True)
gdf.head()

,FID,LSOA Name,LSOA21NMW,BNG_E,BNG_N,LAT,LONG,Shape__Are,Shape__Len,GlobalID,geometry,LAD Name,Deprived %
LSOA Code,,,,,,,,,,,,,
E01000001,1,City of London 001A,None,532123,181632,51.51817,-0.097150,129865.314476,2635.767993,c625aea8-6d73-4b2a-be76-4d5c44cad9f8,"POLYGON ((-0.09667 51.52027, -0.09666 51.52025...",City of London,34.6
E01000002,2,City of London 001B,None,532480,181715,51.51883,-0.091970,228419.782242,2707.816821,52c878e9-ac68-4886-b4a8-fea9cd241a70,"POLYGON ((-0.08969 51.52069, -0.08973 51.52057...",City of London,34.2
E01000003,3,City of London 001C,None,532239,182033,51.52174,-0.095330,59054.204697,1224.573160,b9d8faca-d489-478d-8ce6-acaf76186d7d,"POLYGON ((-0.09653 51.52295, -0.09647 51.52282...",City of London,52.0
E01000005,4,City of London 001E,None,533581,181283,51.51469,-0.076280,189577.709503,2275.805344,15e1417d-537c-4845-9820-fc7596bd59b0,"POLYGON ((-0.07571 51.51575, -0.07542 51.51555...",City of London,66.0
E01000006,5,Barking and Dagenham 016A,None,544994,184274,51.53875,0.089317,146536.995750,1966.092607,8a6c4ee0-c0ff-4736-9cfa-fb12a6d50da0,"POLYGON ((0.09122 51.53905, 0.09148 51.5389, 0...",Barking and Dagenham,63.2


In [12]:
def LAD_Plotter(gdf, LAD_Name=str):
    lsoas_in_lad = gdf[gdf['LAD Name'] == LAD_Name]
    m = lsoas_in_lad.explore(column='Deprived %', cmap='RdYlGn_r', legend=True)
    folium.LayerControl().add_to(m)
    return m

LAD_Plotter(gdf, 'Ealing')